In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Notebook magic

In [4]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

In [5]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-pyboard')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["../micropython/ports/unix/build-2/micropython-2", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    printf('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        if args.pyboard:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

## pyboard

In [57]:
import pyboard
pyb = pyboard.Pyboard('/dev/ttyACM0')
pyb.enter_raw_repl()

In [9]:
pyb.exit_raw_repl()
pyb.close()

In [ ]:
%%micropython -pyboard 1

import utime
import ulab as np

def timeit(n=1000):
    def wrapper(f, *args, **kwargs):
        func_name = str(f).split(' ')[1]
        def new_func(*args, **kwargs):
            run_times = np.zeros(n, dtype=np.uint16)
            for i in range(n):
                t = utime.ticks_us()
                result = f(*args, **kwargs)
                run_times[i] = utime.ticks_diff(utime.ticks_us(), t)
            print('{}() execution times based on {} cycles'.format(func_name, n, (delta2-delta1)/n))
            print('\tbest: %d us'%np.min(run_times))
            print('\tworst: %d us'%np.max(run_times))
            print('\taverage: %d us'%np.mean(run_times))
            print('\tdeviation: +/-%.3f us'%np.std(run_times))            
            return result
        return new_func
    return wrapper

def timeit(f, *args, **kwargs):
    func_name = str(f).split(' ')[1]
    def new_func(*args, **kwargs):
        t = utime.ticks_us()
        result = f(*args, **kwargs)
        print('execution time: ', utime.ticks_diff(utime.ticks_us(), t), ' us')
        return result
    return new_func

__END_OF_DEFS__

# PID module

`ulab` provides a module with a generic and fast implementation of the proportional-integral-derivative ([PID](https://en.wikipedia.org/wiki/Proportional%E2%80%93integral%E2%80%93derivative_controller)) controller. While it is relatively easy to write a PID controller in `python`, the problem with that approach is execution speed: even in thermal applications, where the time scales are not particularly short, running more than one controller simultanously might be difficult, but if the goal is e.g. to mechanically control a self-balancing robot, the task will prove outright impossible. The reason for this slowness of the code execution is the number of floating point operations, where the type-agnostic nature of `python` requires constant type conversion.

`ulab`'s `PID` module implements the controller in C, and for very time-critical applications, it allows the user to take data directly from the measurement
device (most probably an analogue-to-digital converter), if it supports the buffer protocol. Likewise, data can be output directly to the physical device (in most cases a digital-to-analogue converter), if it can take data from a buffer. This direct access to the device data eliminates the need for data conversion in `python`, and reduces the number of objects that must be passed to the controller loop. In fact, in most cases the controller can be run without moving any data whatsoever, simply by calling a method of the `PID` instance without arguments.

## Background

In general, running a controller loop requires the following steps:

1. Define desired set point
1. Measure the current value of the quantity that is to be controlled
1. Calculate the error, the difference between set point and current value
1. Based on the error's value, calculate the output; this is where the control loop itself is defined
1. Place the calculated output on a physical output device
1. Repeat indefinitely

While the description above seems innocuous, steps 2-5 might require considerable resources, when implemented in an interpreted language.

### Measuring the value of a physical quantity

The quantity that a physical device can measure is not necessarily the quantity that we are interested in, which means that the measured quantity has to undergo some conversion steps before it can be used in the controller loop. 

Take the example of temperature measurements: while there are digital thermometers that will output the temperature directly, in most cases, the temperature is only inferred from a voltage. In the simplest case, a thermistor will form half of a voltage divider connected to a reference voltage $U_{ref}$, and the potential $U_{meas}$ is measured at mid-point. It is obvious that even if the thermistors temperature-resistance relationship $R_{th}(T)$ were to be linear, the measured potential will not be (if the thermistor is in the lower half of the divider, $R$ and $R_{th}$ will have to be swapped in the expression below, but that does not alter the fact that $U_{meas}$ is non-linear in $R_{th}$):

\begin{equation}
U_{meas} = U_{ref}\frac{R}{R + R_{th}(T)}
\end{equation}

Most thermistors will have a highly non-linear characteristic curve, thus, converting the potential values to temperature becomes non-trivial, and in the most general case, the conversion must be calculated from the expression above, by inverting $R_{th}(T)$. We can note, however, that any well-behaved function can be expanded in a Taylor series around a point, where the series converges, hence the temperature can be expressed as 

\begin{equation}
T = f(U_{meas}) = 
f(U_0) + 
\left. \frac{df}{dU} \right|_{U_0} (U_{meas} - U_0) + 
\left. \frac{1}{2}\frac{d^2f}{dU^2} \right|_{U_0} (U_{meas} - U_0)^2 + 
\left. \frac{1}{6}\frac{d^3f}{dU^3} \right|_{U_0} (U_{meas} - U_0)^3 + ...
\end{equation}
where the differentials are evaluated at $U_0$.


The coefficients of the expansion are determined by the parameters of the physical system in question, as well as $U_0$. Also note that higher-order expansions are inherently more accurate, but take more time to calculate. Usually, a second-order expansion, or even a linear one is enough, if the expected deviations from $U_0$ are not too large. However, for obvious reasons, the series representation of $f(U_{meas})$ must be monotonic in the domain of interest.

As pointed out, what is actually of interest to us is not the measured values as a function of the environmental value, but the inverse function: we want to know what the environmental value was, if we happen to measure a potential. In the most general case, finding the Taylor series expansion of the inverse function is highly non-trivial. Ben Coleman gives a very good exposition of the [subject](https://randorithms.com/2021/08/31/Taylor-Series-Inverse.html). But it might not be required to go down that path: given a small number of values of the $T(U_{meas})$ relationship, we might get away with a polynomial fit, and use that in lieu of the Taylor series. This must be done in terms of $(U_{meas} - U_0)$, however.

### Converting the digital control values to physical control signals

The comments on converting measured values to physically relevant quantities apply, when we want to convert calculated values to physially relevant control signals, i.e., when at the end of the control loop, we want to use the calculated signal to influence the physical drive mechanism: the calculated digital value will be converted to a current, potential etc. by means of a digital-to-analogue converter (DAC), and some analogue circuitry that takes the analogue output potential of the DAC as input, and generates a current, another potential as output, or turns a valve. The relationship between the input potential and the physical control quantity is not necessarily the identity, or even linear, thus, in the general case, we have to express the conversion function as 

\begin{equation}
I_{control} = g(U_{control}) = 
g(U^*) + 
\left. \frac{dg}{dU} \right|_{U^*} (U_{control} - U^*) + 
\left. \frac{1}{2}\frac{d^2g}{dU^2} \right|_{U^*} (U_{control} - U^*)^2 + 
\left. \frac{1}{6}\frac{d^3g}{dU^3} \right|_{U^*} (U_{control} - U^*)^3 + ...
\end{equation}
where the control output is expanded around the potential $U^*$. 

The coefficients of the expansion are determined by the parameters of the physical system in question, as well as $U^*$. Also note that higher-order expansions are inherently more accurate, but take more time to calculate. Usually, a second-order expansion, or even a linear one is enough, if the expected deviations from $U^*$ are not too large. Again, we do not necessarily have to deal with the Taylor series, and it might suffice to use a polynomial fit to some pre-calculated values. If that is the case, the expansion must be in terms of $(U_{control} - U^*)$.

### Input-output buffers

Conversion between analogue and digital values works not with floating point numbers, which are most probably the results of the controller algorithm, but with integers that are transmitted to/from the converter. A typical transmission sequence will, in general, contain not only the data, but some auxiliary bits, like status, channel number, gain, etc., as shown below. The input/output string $S$ consists of $n + N + m$ bits, where the *D*s are the data bits with the digital values, while the *A*s are the auxiliary bits that are important for the functioning of the converter, but carry no numerical information. The data relevant to the numerical control algorithm are embedded in a string, whose auxiliary bits are fixed, and do not change from transmission to transmission.

\begin{equation}
S = \overbrace{A_n A_{n - 1} ... A_1 A_0}^{\mathrm{auxiliary\ bits}}
\overbrace{D_N D_{N - 1} ... D_1 D_0}^{\mathrm{data\ bits}} 
\overbrace{A_m A_{m - 1} ... A_1 A_0}^{\mathrm{auxiliary\ bits}}
\end{equation}

In order to make data exchange between the control algorithm and the peripheral devices transparent and seamless, the `PID` module implements input/output buffers, which not only place the relevant bits at the correct positions in pre-defined strings automatically, but also take care of the conversion of data bits to floating point numbers and vica versa.

Since we are not interested in the sequences $A_n A_{n - 1} ... A_1 A_0$, and $A_m A_{m - 1} ... A_1 A_0$, the data can be recovered by shifting its bits to the right by so many places that $D_0$ reaches a byte boundary, and then masking the result by $2^N - 1$ (this is equivalent to $N$ 1s). In the code, $n$ will be referred to as the `offset`, while $N$ as the `bitdepth`. The exact values should be taken from the data sheets of the ADC or DAC that are used as the interface devices.

### Input-output data converters

We have seen above how the numbers in the control algorithm can be interfaced with the outside world. To facilitate the convension, `ulab`'s `PID` module defines input-output data buffers, which hold an input/output buffer as described in the previous section, and the coefficients of the Taylor series expansions, i.e., all information required to turn a string of bits into a floating point number that is the true represenation of the value that we are interested in.

## Methods 

The module implements a class with the following methods

1. [reset](#reset)
1. [step](#step)
1. [float_step](#float_step)

Parameters and some internal values can be accessed via the following attributes of an instance:

1. [P](#P)
1. [I](#I)
1. [D](#D)
1. [setpoint](#setpoint)
1. [value](#value)
1. [input/output](#buffers)

### Instantiating the PID object

The controller can be instantiated by importing the `PID` module, and calling the constructor without arguments. At this point, the object will hold all values required for the operation of the PID loop, and some of these can be displayed by simply printing the controller object: 

In [17]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print(pid)

PID object at 0x7f94172053a0




### reset

This method takes no arguments, and resets the controller, i.e., zeroes the time, the number of steps, and the integral part.

In [45]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
pid.P = 1.0
pid.I = 0.1
pid.setpoint = 0.0

print('initial state of controller')
print(f'value: {pid.value}, steps: {pid.steps}')

print()
print('stepping the controller 10 times')
for i in range(10):
    pid.float_step(3.0 / (i + 1))

print(f'value: {pid.value}, out: {pid.out}, steps: {pid.steps}')

print()
print('resetting controller')
pid.reset()
print(f'value: {pid.value}, out: {pid.out}, steps: {pid.steps}')

initial state of controller
value: 0.0, steps: 0

stepping the controller 10 times
value: 0.3, out: 1.178690476190476, steps: 10

resetting controller
value: 0.0, out: 0.0, steps: 0




### step

This method takes no arguments, and steps the controller, i.e., takes and converts the data in the inpput buffer, calculates the three terms of the controller's output, and places the data on the output buffer. At the same time, the `value`, `steps`, and `integral` attributes of the controller are updated.

In [58]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()

# initialise the input/output buffers with default values
pid.input.init()
pid.output.init()

pid.P = 1.0
pid.D = 1.0

pid.step()
print(f'value: {pid.value}, out: {pid.out}')

None
value: 0.0, out: 0.0




### float_step

The method takes one, or two arguments, and returns the computed control value. The first argument is the current value of the signal to be controlled, while the time can be supplied in the optional second argument. If that is missing, then a time increment of 1.0 is taken as default. As with [step](#step), all internal variables are updated.

In [55]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
pid.P = 1.0
pid.D = 1.0
print(f'P: {pid.P}, I: {pid.I}, D: {pid.D}')

print(pid.float_step(1.0))
print(pid.float_step(2.0, 0.2))

P: 1.0, I: 0.0, D: 1.0
2.0
0.75




### P

This attribute *gets* or *sets* the constant in front of the term proportinal to the error. In the first case, it returns a floating point number, in the second case, a floating point number can be assigned to it.

In [40]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print(f'P: {pid.P}')
pid.P = 13.0
print(f'P: {pid.P}')


P: 0.0
P: 13.0




### I

This attribute *gets* or *sets* the constant in front of the term proportinal to the integral of the error. In the first case, it returns a floating point number, in the second case, a floating point number can be assigned to it.

In [43]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print(f'I: {pid.I}')
pid.I = 1.0
print(f'I: {pid.I}')

I: 0.0
I: 1.0




### D

This attribute *gets* or *sets* the constant in front of the term proportinal to the derivative of the error. In the first case, it returns a floating point number, in the second case, a floating point number can be assigned to it.

In [44]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print(f'D: {pid.D}')
pid.D = 0.5
print(f'D: {pid.D}')

D: 0.0
D: 0.5




## Buffers

As mentioned above, the controller interfaces with external devices through its input/output buffers. Internally, these hold the parameters of the Taylor expansion, and everything required to turn the bits of the binary stream of the ADC to floating point numbers, and to turn the floating point numbers into the binary stream consumed by the DAC.

The `PID` instance' `input` and `output` attributes return a `buffer` object that has the method

1. [evaluate](#evaluate)
1. [init](#init)

and the following attributes:

1. [bitdepth](#bitdepth)
1. [bytes](#bytes)
1. [coeffs](#coeffs)
1. [mask](#mask)
1. [offset](#offset)
1. [order](#order)
1. [x0](#x0)

### evaluate

This method evaluates the series expansion at the argument `x - x0`, and returns a floating point number with the computed value.

In [54]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
pid.input.init()
# y = x, x0 = 0.0, default
print(f'coefficients: {pid.input.coeffs}, x0: {pid.input.x0}')
print(f'value at x = 10.0: {pid.input.evaluate(10.0)}')

print()
pid.input.x0 = 1.0
# y = (x - x0) ** 2 + 2 * (x - x0) + 3.0
pid.input.coeffs = (1, 2, 3)
print(f'coefficients: {pid.input.coeffs}, x0: {pid.input.x0}')
# x - x0 = 9 -> y = 81 + 2 * 9 + 3
print(f'value at x = 10.0: {pid.input.evaluate(10.0)}')


coefficients: (1.0, 0.0), x0: 0.0
value at x = 10.0: 10.0

coefficients: (1.0, 2.0, 3.0), x0: 1.0
value at x = 10.0: 102.0




### init

The method initialises the input's ADC or the output's DAC buffer with default values, i.e., it sets [x0](#x0) to 0.0, and [coeffs](#coeffs) to the identity, i.e., to the array `[1.0, 0.0]`.

In [26]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print(f'x0: {pid.input.x0}, order: {pid.input.order}, coefficients: {pid.input.coeffs}')
pid.input.init()
print(f'x0: {pid.input.x0}, order: {pid.input.order}, coefficients: {pid.input.coeffs}')

x0: 0.0, order: 0, coefficients: ()
x0: 1.0, order: 2, coefficients: (1.0, 0.0)




### bitdepth

This attribute *gets* or *sets* the bit depth of the input's ADC or the output's DAC. Note that setting the value of the attribute does not communicate with the ADC or DAC, it simply changes how the bits in the input and output buffers are interpreted. If the attribute is used as a getter, it returns the currently active bitdepth setting for that buffer. If an unsigned integer is assigned to the attribute (i.e., when it is a *setter*) the number of bits taken by the data in the buffer will be set. 

In [8]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print('initial bit depth: ', pid.input.bitdepth)
pid.input.bitdepth = 12
print('new bit depth: ', pid.input.bitdepth)

initial bit depth:  0
new bit depth:  12




### bytes

This attribute *gets* or *sets* the byte array of the input's ADC or the output's DAC. Note that setting the value of the attribute does not communicate with the ADC or DAC, it simply changes the pointer to the memory segment. If the attribute is used as a getter, it returns the currently active byte array setting for that buffer. If a byte array is assigned to the attribute (i.e., when it is a *setter*) that byte array will be set to the corresponding internal variable.

In [9]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
buffer = bytearray([1, 2, 3, 4])
print('bytearray: ', buffer)
pid.input.bytes = buffer
print('bytearray in PID object: ', pid.input.bytes)

bytearray:  bytearray(b'\x01\x02\x03\x04')
bytearray in PID object:  bytearray(b'\x01\x02\x03\x04')




### coeffs

This attribute *gets* or *sets* the coefficients of the Taylor series expansion. Without assignment, the attribute returns a tuple of floating point numbers in descending order of the powers. The attribute can be used as a *setter*, when a generic `micropython` interable is assigned to it. In addition to converting the items in the iterable to floating point numbers, the setter also assigns the internal variable [order](#order).

In [10]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
pid.input.coeffs = [1, 2, 3]
print('coefficients: ', pid.input.coeffs)

coefficients:  (1.0, 2.0, 3.0)




### mask

This attribute only *gets* the computed internal variable `mask`, which is updated, whenever the [bitdepth](#bitdepth) is set.

In [13]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
pid.input.bitdepth = 12
print('bit depth: ', pid.input.bitdepth)
print('mask: ', pid.input.mask)


bit depth:  12
mask:  4095




### offset

This attribute *gets* or *sets* the offset of the input's ADC or the output's DAC stream. Note that setting the value of the attribute does not communicate with the ADC or DAC, it simply changes the pointer to the memory segment. f the attribute is used as a getter, it returns the currently active offset setting for that buffer. If an unsigned integer is assigned to the attribute (i.e., when it is a *setter*) the buffer's internal offset variable will be set.

In [14]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print('offset: ', pid.input.offset)
pid.input.offset = 12
print('new offset: ', pid.input.offset)

offset:  0
new offset:  12




### order

This attribute only *gets* the computed internal variable `order`, which is inferred from the length of the Taylor series expansion's setting in [coeffs](#coeffs).

In [15]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
pid.input.coeffs = [1, 2, 3]
print('coefficients: ', pid.input.coeffs)
print('expansion order: ', pid.input.order)

coefficients:  (1.0, 2.0, 3.0)
expansion order:  3




### x0

This attribute *gets* or *sets* the point around which the ADC's input, or the DAC's output is expanded. If the attribute is used as a getter, it returns the current value, while if a floating point number is assigned to it (i.e., when it is a *setter*) expansion point will be set.

In [16]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print('x0: ', pid.input.x0)
pid.input.x0 = 123.0
print('new x0: ', pid.input.x0)

x0:  0.0
new x0:  123.0


